# Progetto Computer Science

*Nome:* Christian

*Cognome:* Uccheddu

*Matricola:* 800428


**Setup dell'ambiente**

Importazione di tutte le librerie utili al funzionamento del programma e settaggio dei path in cui sono presenti i dati in modo da essere facilmente cambiabili all'inizio del programma.

In [1]:
import pandas as pd
import datetime
import numpy as np
from IPython.display import display

In [2]:
loans_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans.csv"
loans_lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans_lenders.csv"
lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/lenders.csv"
country_stats_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/country_stats.csv" 

**1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.**

Per questo compito mi occupo di importare il file loans_lenders, di analizzarne la struttura e successivamente decidere come procedere.

In [3]:
loans_lenders = pd.read_csv(loans_lenders_path)

In [4]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


Vedendo la struttura del dataframe, la normalizzazione della tabella consiste nel separare gli elementi presenti nella colonna lenders in modo che il dataframe sia formato da celle in cui è presente solo una istanza. Per farlo separo gli elementi che si trovano in corrispondenza della virgola, essendoci uno spazio vuoto dopo la virgola dovrò stare attento a considerarlo. Per ovviare a questo problema trasformo gli elementi ad ogni istanza di lenders in una lista.

In [6]:
loans_lenders['lenders'] = loans_lenders.apply(lambda row: row['lenders'].split(','), axis = 1)

Per separare gli elementi utilizzo la funzione *explode* implementata all'interno della libreria pandas che permette di sepoarare gli elementi di una lista lungo l'asse voluto.

In [7]:
loans_lenders_norm = loans_lenders.explode('lenders')

In [13]:
# Decommentare questa riga per vedere i risultati.
#loans_lenders_norm.head()

In [ ]:
display(loans_lenders_norm.head())